<a href="https://colab.research.google.com/github/kellemes13/Tool/blob/main/Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np



# Define a function to read data from a file and plot a graph
def plot_bw(title, file_path):
    # Read data from file
    with open(file_path, 'r') as file:
        data = file.readlines()


    # Process data
    x = []
    y = []
    i=1
    for line in data:
        values = line.split(',')
        y.append(int(values[1])/1000*20)
        x.append(i)
        i=i+1


    # Plot the graph
    plt.plot(x, y,'b')
    plt.grid(True)
    plt.xlabel('time(sec)')
    plt.ylabel('MB/s')
    plt.title('Bandwidth: ' + title)
    plt.show()


    # Define a function to read data from a file and plot a graph
def plot_iops(title,file_path):
    # Read data from file
    with open(file_path, 'r') as file:
        data = file.readlines()


    # Process data
    x = []
    y = []
    i=1
    for line in data:
        values = line.split(',')
        y.append(int(values[1])*20)
        x.append(i)
        i=i+1

    y_kiops = [element / 1000 for element in y]


    # Plot the graph
    plt.plot(x, y_kiops, 'b')
    plt.grid(True)
    plt.xlabel('time(sec)')
    plt.ylabel('IO')
    plt.title('KIOPS: '+title)
    plt.show()


# Define a function to read data from a file and plot a graph
def plot_clat(title, file_path):
    # Read data from file
    with open(file_path, 'r') as file:
        data = file.readlines()


    # define a figure
    plt.figure()

    # Process data
    x = []
    y = []

    for line in data:
        values = line.split(',')
        y.append(int(values[1])/1000/20)
        x.append(len(y))

    y_slot = sorted(y)
    x_percentage = [element / len(x)*100 for element in x]

    # plot graph
    s_pt99 = list(map(lambda k: k >= 99, x_percentage)).index(True)
    s_pt999 = list(map(lambda k: k >= 99.9, x_percentage)).index(True)
    s_pt9999 = list(map(lambda k: k >= 99.99, x_percentage)).index(True)

    l = len(x_percentage)
    x_spt = int(l*0.25)
    x_0_99 = np.linspace(0, x_spt, s_pt99)
    x_99_999 = np.linspace(x_spt+1, x_spt*2, s_pt999-s_pt99)
    x_999_9999 = np.linspace(x_spt*2+1, x_spt*3, s_pt9999-s_pt999)
    x_9999_end = np.linspace(x_spt*3+1, l-1, l-s_pt9999)

    y_0_99 = y_slot[:s_pt99]
    y_99_999 = y_slot[s_pt99:s_pt999]
    y_999_9999 = y_slot[s_pt999:s_pt9999]
    y_9999_end = y_slot[s_pt9999:]

    #plot line
    plt.plot(x_0_99, y_0_99, 'b')
    plt.plot(x_99_999, y_99_999, 'b')
    plt.plot(x_999_9999, y_999_9999, 'b')
    plt.plot(x_9999_end, y_9999_end, 'b')
    #smooth line
    plt.plot([x_0_99[-1], x_99_999[0]], [y_0_99[-1], y_99_999[0]], 'b')
    plt.plot([x_99_999[-1], x_999_9999[0]], [y_99_999[-1], y_999_9999[0]], 'b')
    plt.plot([x_999_9999[-1], x_9999_end[0]], [y_999_9999[-1], y_9999_end[0]], 'b')
    #mark pointview
    plt.plot(x_99_999[0],y_99_999[0],'ro')
    plt.plot(x_999_9999[0],y_999_9999[0],'ro')
    plt.plot(x_9999_end[0],y_9999_end[0],'ro')
    #mark number
    x_loc=1.06
    y_loc=0.95
    plt.text(x_99_999[0]*x_loc, y_99_999[0]*y_loc,'{}us'.format(round(y_99_999[0], 2)), weight='bold')
    plt.text(x_999_9999[0]*x_loc, y_999_9999[0]*y_loc,'{}us'.format(round(y_999_9999[0], 2)), weight='bold')
    plt.text(x_9999_end[0]*x_loc, y_9999_end[0]*y_loc,'{}us'.format(round(y_9999_end[0], 2)), weight='bold')

    plt.xticks([0, x_spt, x_spt*2, x_spt*3, l-1], [0, 99, 99.9, 99.99, 100])
    plt.grid(True)
    plt.xlabel("QoS (%)")
    plt.ylabel('us')
    plt.title('Latency: '+ title)
    plt.show()

# sequential write
plot_bw('sw','/content/sw_bw.1.log')
# random write
plot_iops('rw','/content/rw_iops.1.log')
plot_clat('rw','/content/rw_clat.1.log')
# sequential read
plot_bw('sr','/content/sr_bw.1.log')
# random read
plot_iops('rr','/content/rr_iops.1.log')
plot_clat('rr','/content/rr_clat.1.log')